## Setting up gemini

https://ai.google.dev/

https://aistudio.google.com/app/apikey


In [1]:
import google.generativeai as genai

genai.configure(api_key="")
model = genai.GenerativeModel('gemini-pro')

/Users/yuuki/Documents/Github/Workshop/intro-to-llm-app/demo/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
response = model.generate_content("Can you say hello?")
print(response.parts)

## Build basic Chat processing interface

In [ ]:
import gradio as gr

In [ ]:
test = """
Subject: Recap of Expenses from Our Recent Party

I hope this email finds you well. I wanted to take a moment to thank you for joining us at the party last week. It was fantastic having you there, and your presence truly added to the fun atmosphere!

I've gone through the expenses from the party, and I wanted to provide you with a breakdown of the costs incurred. Here are the five main items along with their respective costs:

Chips: $20
Coca-cola: $50
Pepsi: $40
Pretzel: $40
Popcorn: $60

Since we both participated in the event, I thought it would be helpful to share these details with you.

If you have any questions or concerns about the expenses, please don't hesitate to reach out. I believe in transparency when it comes to sharing costs, and I want to ensure that everything is clear and accounted for.

Once again, thank you for being a part of the celebration. I had a wonderful time, and I hope you did too!

Looking forward to hearing from you soon.

Best regards,

Alice
"""

In [ ]:
def extract(inp):
  prompt = f"""
  You are an efficient accountant assistant.
  Your job is to read the input text list out the item in the test with the following csv format

  item-name,cost

  Here is the INPUT:

  {inp}

  OUTPUT:
  """

  response = model.generate_content(prompt)
  return response.text

In [ ]:
def save_to_csv(text):
  filename = "result.csv"
  lines = [line.strip() for line in text.split("\n") if line.strip()]
  with open(filename, 'w') as csvfile:
    for line in lines:
      csvfile.write(line + "\n")
    csvfile.close()

In [ ]:
with gr.Blocks() as demo:
  inp = gr.Textbox(value=test)
  out = gr.Textbox(label="Output")
  with gr.Row():
    run_btn = gr.Button("Run")
    save_btn = gr.Button("Save")
  run_btn.click(fn=extract, inputs=inp, outputs=out)
  save_btn.click(fn=save_to_csv, inputs=out)
  print(out.value)

demo.launch()

## Chaining of agent

In [ ]:
def extract_agent(inp):
  prompt = f"""
  You are an efficient accountant assistant.
  Your job is to read the input text list out the item in the test with the following csv format

  item-name,cost

  Represent the cost as pure number, with no extra symbol.

  Here is the INPUT:

  {inp}

  OUTPUT:
  """

  response = model.generate_content(prompt)
  return response.text

In [ ]:
def classify_agent(inp):
  prompt = f"""
  You are an efficient classifier assistant.
  There are two classification categories: "drink" and "snack".
  Classsify the following input into this csv format:

  category, item-name, cost

  Here is the INPUT:

  {inp}

  OUTPUT:
  """

  response = model.generate_content(prompt)
  return response.text

In [ ]:
def summarize_agent(inp):
  prompt = f"""
  You are an efficient summarizer.
  Your task is to sum up the total cost of every entry of the same category of the following input.
  The input will has the following format:

  category, item-name, cost

  Return the ouput as follow:

  category, total-cost

  Here is the INPUT:

  {inp}

  OUTPUT:
  """

  response = model.generate_content(prompt)
  return response.text

In [ ]:
def agent_chain(text):
  extract = extract_agent(text)
  print(f"Extract: \n {extract} \n")
  classify = classify_agent(extract)
  print(f"Classify: \n {classify} \n")
  summarize = summarize_agent(classify)

  return summarize

In [ ]:
import pandas as pd

def plot(inp):
  frame = {
    "Category": [],
    "Total cost ($)": []
  }

  for line in inp.strip().split("\n"):
    [cat, cost] = line.strip().split(",")
    frame["Category"].append(cat.strip())
    frame["Total cost ($)"].append(cost.strip())

  print(frame)

  return  gr.BarPlot(
    pd.DataFrame(frame),
    x="Category",
    y="Total cost ($)",
    title="Breakdown of party cost",
    min_width=300
  )
      
  

In [ ]:
with gr.Blocks() as demo:
  inp = gr.Textbox(value=test)
  out = gr.Textbox(label="Output")
  with gr.Row():
    run_btn = gr.Button("Run")
    save_btn = gr.Button("Save")
    plot_btn = gr.Button("Plot")
  plt = gr.BarPlot()

  run_btn.click(fn=agent_chain, inputs=inp, outputs=out)
  save_btn.click(fn=save_to_csv, inputs=out)
  plot_btn.click(fn=plot, inputs=out, outputs=plt)
  print(out.value)

demo.launch()